In [2]:
import sqlalchemy as db

In [3]:
db_name = 'db_files/temp_db.sqlite3'

In [6]:
engine = db.create_engine(f'sqlite:///{db_name}')

In [84]:
conn = engine.connect()

Доп таблицы для Москвы и Еревана

In [128]:
text_query_create_employee_moscow = f"""CREATE TABLE employee_moscow
(id INTEGER PRIMARY KEY,
name VARCHAR(128) NOT NULL,
lastname VARCHAR(128) NOT NULL,
salary INTEGER CHECK(salary >= 0) DEFAULT 0)"""

text_query_create_employee_yerevan = f"""CREATE TABLE employee_yerevan
(id INTEGER PRIMARY KEY,
name VARCHAR(128) NOT NULL,
lastname VARCHAR(128) NOT NULL,
salary INTEGER CHECK(salary >= 0) DEFAULT 0)"""

In [131]:
# conn.execute(db.text(text_query_create_employee_yerevan))

In [132]:
conn.execute(db.text(text_query_create_employee_moscow))

In [133]:
def set_default_moscow(conn):
    data = """| 1  | Thomas | Nelson     | 150000 |
| 2  | Willie | Mitchell   | 90000  |
| 3  | David  | Garrett    | 170000 |
| 4  | Gary   | Richardson | 140000 |
| 5  | Donald | Stewart    | 100000 |"""
    keys = ('id', 'name', 'lastname', 'salary')
    default_data = [dict(zip(keys, map(str.strip, line.strip('|').split('|')))) 
                   for line in data.splitlines()]
    query_set = db.insert(db.Table('employee_moscow', db.MetaData(), autoload_with=engine)).values(default_data)
    query_delete = db.delete(db.Table('employee_moscow', db.MetaData()))
    conn.execute(query_delete)
    # conn.commit()
    conn.execute(query_set)
    conn.commit()    

In [134]:
def set_default_yerevan(conn):
    data = """| 1  | Gary      | Richardson | 140000 |
| 2  | Donald    | Stewart    | 100000 |
| 3  | Frederick | White      | 70000  |
| 4  | Willie    | Gonzales   | 12000  |
| 5  | Angel     | Carter     | 990000 |"""
    keys = ('id', 'name', 'lastname', 'salary')
    default_data = [dict(zip(keys, map(str.strip, line.strip('|').split('|')))) 
                   for line in data.splitlines()]
    query_set = db.insert(db.Table('employee_yerevan', db.MetaData(), autoload_with=engine)).values(default_data)
    query_delete = db.delete(db.Table('employee_yerevan', db.MetaData()))
    conn.execute(query_delete)
    # conn.commit()
    conn.execute(query_set)
    conn.commit()    
    

Напишите запрос, который создаст таблицу employee с описанными ниже полями. Обратите внимание на ограничения (constraints).

id - целое число (первичный ключ)
name - строка с длинной 128 (не null)
lastname - строка с длинной 128 (не null)
salary - (больше либо равно нулю, ноль по умолчанию)

In [21]:
stmt_create_table = db.Table(
    'employee',
    db.MetaData(), 
    db.Column('id', db.INTEGER(), primary_key=True),
    db.Column('name', db.VARCHAR(128), nullable=False),
    db.Column('lastname', db.VARCHAR(128), nullable=False),
    db.Column('salary', db.INTEGER(), db.CheckConstraint('salary >= 0'), default=0)
)

In [22]:
compiled_query = stmt_create_table.compile()

In [25]:
compiled_query.sql_compiler.string

''

In [11]:
text_query_create_employee = f"""CREATE TABLE employee
(id INTEGER PRIMARY KEY,
name VARCHAR(128) NOT NULL,
lastname VARCHAR(128) NOT NULL,
salary INTEGER CHECK(salary >= 0) DEFAULT 0)"""

In [7]:
query_show_tables = "SELECT name FROM sqlite_master WHERE type='table';"

In [139]:
conn.execute(db.text(query_show_tables)).fetchall()
# with engine.connect() as conn:
#     res = conn.execute(db.text(query_show_tables))
# print(res.fetchall())

[('employee',), ('employee_yerevan',), ('employee_moscow',)]

In [12]:
# with engine.connect() as conn:
#     res = conn.execute(db.text(text_query_create_employee))
# print(res)

In [13]:
# with engine.connect() as conn:
#     res = conn.execute(db.text(query_show_tables))
# print(res.fetchall())

[('employee',)]


В базе существует таблица employee с полями:

id - целое число (первичный ключ)
name - строка с длинной 128 (не null)
lastname - строка с длинной 128 (не null)
salary - (больше либо равно нулю, ноль по умолчанию)

Напишите SQL скрипт, который добавляет 8 записей с сотрудниками (данные произвольные).

In [79]:
# очистка таблицы

# with engine.connect() as conn:
#     conn.execute(db.delete(db.Table('employee', db.MetaData())))
#     conn.commit()

In [49]:
# подготовка тестовых данных
from random import randint
names = [x * 3 for x in 'abcdefgh']
lastnames = [x * 8 for x in 'abcdefgh'[::-1]]
salaries = [randint(100, 500) for _ in range(8)]

In [50]:
# 
items = [{'id': i, "name": name, 'lastname': lastname, 'salary': salary} 
         for i, (name, lastname, salary) in enumerate(zip(names, lastnames, salaries), 1)]
items

[{'id': 1, 'name': 'aaa', 'lastname': 'hhhhhhhh', 'salary': 274},
 {'id': 2, 'name': 'bbb', 'lastname': 'gggggggg', 'salary': 178},
 {'id': 3, 'name': 'ccc', 'lastname': 'ffffffff', 'salary': 265},
 {'id': 4, 'name': 'ddd', 'lastname': 'eeeeeeee', 'salary': 348},
 {'id': 5, 'name': 'eee', 'lastname': 'dddddddd', 'salary': 339},
 {'id': 6, 'name': 'fff', 'lastname': 'cccccccc', 'salary': 196},
 {'id': 7, 'name': 'ggg', 'lastname': 'bbbbbbbb', 'salary': 286},
 {'id': 8, 'name': 'hhh', 'lastname': 'aaaaaaaa', 'salary': 360}]

In [88]:
# данные из курса
keys = ('id', 'name', 'lastname', 'salary')
data = """| 1  | Thomas    | Nelson     | 150000 |
| 2  | Willie    | Mitchell   | 90000  |
| 3  | David     | Garrett    | 170000 |
| 4  | Gary      | Richardson | 140000 |
| 5  | Donald    | Stewart    | 100000 |
| 6  | Frederick | White      | 70000  |
| 7  | Willie    | Gonzales   | 12000  |
| 8  | Angel     | Carter     | 990000 |"""
course_data = [dict(zip(keys, map(str.strip, line.strip('|').split('|')))) for line in data.splitlines()]

In [89]:
course_data

[{'id': '1', 'name': 'Thomas', 'lastname': 'Nelson', 'salary': '150000'},
 {'id': '2', 'name': 'Willie', 'lastname': 'Mitchell', 'salary': '90000'},
 {'id': '3', 'name': 'David', 'lastname': 'Garrett', 'salary': '170000'},
 {'id': '4', 'name': 'Gary', 'lastname': 'Richardson', 'salary': '140000'},
 {'id': '5', 'name': 'Donald', 'lastname': 'Stewart', 'salary': '100000'},
 {'id': '6', 'name': 'Frederick', 'lastname': 'White', 'salary': '70000'},
 {'id': '7', 'name': 'Willie', 'lastname': 'Gonzales', 'salary': '12000'},
 {'id': '8', 'name': 'Angel', 'lastname': 'Carter', 'salary': '990000'}]

In [54]:
# alchemy insert with data
stmt_insert_employee = db.insert(db.Table('employee', db.MetaData(), autoload_with=engine)).values(items)

In [58]:
print(stmt_insert_employee.compile(compile_kwargs={"literal_binds": True}).string)

INSERT INTO employee (id, name, lastname, salary) VALUES (1, 'aaa', 'hhhhhhhh', 274), (2, 'bbb', 'gggggggg', 178), (3, 'ccc', 'ffffffff', 265), (4, 'ddd', 'eeeeeeee', 348), (5, 'eee', 'dddddddd', 339), (6, 'fff', 'cccccccc', 196), (7, 'ggg', 'bbbbbbbb', 286), (8, 'hhh', 'aaaaaaaa', 360)


In [90]:
stmt_insert_employee_course_date = db.insert(db.Table('employee', db.MetaData(), autoload_with=engine)).values(course_data)
print(stmt_insert_employee_course_date.compile(compile_kwargs={"literal_binds":True}).string)

INSERT INTO employee (id, name, lastname, salary) VALUES (1, 'Thomas', 'Nelson', 150000), (2, 'Willie', 'Mitchell', 90000), (3, 'David', 'Garrett', 170000), (4, 'Gary', 'Richardson', 140000), (5, 'Donald', 'Stewart', 100000), (6, 'Frederick', 'White', 70000), (7, 'Willie', 'Gonzales', 12000), (8, 'Angel', 'Carter', 990000)


In [80]:
# вносим данные в таблицу
with engine.connect() as conn:
    conn.execute(stmt_insert_employee_course_date)
    conn.commit()

In [27]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
with Session() as session:
    query = session.query(stmt_create_table)
    

В базе существует таблица employee с полями:

id - целое число (первичный ключ)
name - строка с длинной 128 (не null)
lastname - строка с длинной 128 (не null)
salary - (больше либо равно нулю, ноль по умолчанию)

Создайте представление v_employee которое будет содержать имена и фамилии всех сотрудников. 

In [67]:
query_create_view = """CREATE VIEW v_employee AS SELECT name, lastname FROM employee"""

In [71]:
conn.execute(db.text(query_create_view))
conn.commit()
# with engine.connect() as conn:
#     conn.execute(db.text(query_create_view))
#     conn.commit()

В базе существует представление v_employee. Удалите его.

In [69]:
# with engine.connect() as conn:
conn.execute(db.text('DROP VIEW v_employee'))

В базе существует таблица employee. Всем сотрудникам, у которых зарплата не превышает 100000 единиц, укажите зарплату в 120000 единиц.

In [81]:
query_up_salary = """UPDATE employee SET salary=120000 WHERE employee.salary <=100000"""

In [83]:
# with engine.connect() as conn:
conn.execute(db.text(query_up_salary))
conn.commit()

In [102]:
conn.execute(db.text('SELECT * FROM employee')).fetchall()

[(1, 'Thomas', 'Nelson', 150000),
 (2, 'Willie', 'Mitchell', 100000),
 (3, 'David', 'Garrett', 170000),
 (4, 'Gary', 'Richardson', 140000),
 (5, 'Donald', 'Stewart', 100000),
 (6, 'Frederick', 'White', 80000),
 (7, 'Willie', 'Gonzales', 22000),
 (8, 'Angel', 'Carter', 990000)]

В базе существует таблица employee. Увеличьте зарплату на 10000 единиц у всех сотрудников, у которых зарплата меньше 100000 единиц.

In [94]:
query_clean_employee = """DELETE FROM employee"""

In [96]:
conn.execute(db.text(query_clean_employee))
conn.commit()

In [97]:
conn.execute(stmt_insert_employee_course_date)
conn.commit()

In [120]:
def set_employee_to_default(conn):
    data = """| 1  | Thomas    | Nelson     | 150000 |
| 2  | Willie    | Mitchell   | 90000  |
| 3  | David     | Garrett    | 170000 |
| 4  | Gary      | Richardson | 140000 |
| 5  | Donald    | Stewart    | 100000 |
| 6  | Frederick | White      | 70000  |
| 7  | Willie    | Gonzales   | 12000  |
| 8  | Angel     | Carter     | 990000 |"""
    keys = ('id', 'name', 'lastname', 'salary')
    default_data = [dict(zip(keys, map(str.strip, line.strip('|').split('|')))) 
                   for line in data.splitlines()]
    query_set = db.insert(db.Table('employee', db.MetaData(), autoload_with=engine)).values(default_data)
    query_delete = db.delete(db.Table('employee', db.MetaData()))
    conn.execute(query_delete)
    # conn.commit()
    conn.execute(query_set)
    conn.commit()    

In [98]:
query_add_to_salary = """UPDATE employee SET salary=employee.salary + 10000 WHERE employee.salary < 100000"""

In [100]:
conn.execute(db.text(query_add_to_salary))
conn.commit()

В базе существует таблица employee. Удалите всех сотрудников, у которых зарплата меньше 100000 единиц.

In [121]:
set_employee_to_default(conn)

In [122]:
query_del_poverty = """DELETE FROM employee WHERE employee.salary < 100000"""

In [123]:
conn.execute(db.text(query_del_poverty))
conn.commit()

В базе существует две таблицы, которые содержат данные о сотрудниках в Москве и в Ереване (employee_moscow и employee_yerevan). Объединить имена сотрудников из двух твблиц. Вывести все имена, в том числе с повторением.

In [135]:
set_default_moscow(conn)

In [136]:
set_default_yerevan(conn)

In [137]:
query_names = "SELECT name FROM employee_moscow UNION ALL SELECT name FROM employee_yerevan"

In [138]:
conn.execute(db.text(query_names)).fetchall()

[('Thomas',),
 ('Willie',),
 ('David',),
 ('Gary',),
 ('Donald',),
 ('Gary',),
 ('Donald',),
 ('Frederick',),
 ('Willie',),
 ('Angel',)]

В базе существует две таблицы, которые содержат данные о сотрудниках в Москве и в Ереване (employee_moscow и employee_yerevan). Объединить имена сотрудников из двух твблиц. Итоговая выборка должна содержать поля name с именем сотрудника и city со значением Moscow для Москвы и Yerevan для Еревана.

In [140]:
query_name_city = """SELECT name, 'Moscow' as city FROM employee_moscow UNION ALL SELECT name, 'Yerevan' as city FROM employee_yerevan"""

In [141]:
conn.execute(db.text(query_name_city)).fetchall()

[('Thomas', 'Moscow'),
 ('Willie', 'Moscow'),
 ('David', 'Moscow'),
 ('Gary', 'Moscow'),
 ('Donald', 'Moscow'),
 ('Gary', 'Yerevan'),
 ('Donald', 'Yerevan'),
 ('Frederick', 'Yerevan'),
 ('Willie', 'Yerevan'),
 ('Angel', 'Yerevan')]

In [142]:
conn.close()